In [0]:
%pip install unidecode gspread==5.12.4
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
from abc import ABC, abstractmethod
import time
import mlflow
from contextlib import contextmanager
import json
import pandas as pd
import datetime
import re
import openai
from openai import OpenAI
import gspread
import random
import logging
import os
from unidecode import unidecode
from pathlib import Path
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
from functools import reduce
from typing import *
import tiktoken
import io


from localizers.general_config import CENTRALIZED_SHEET_URL, DIR

In [0]:
#Orchestrator

In [0]:
%run "./authenticationScript"

In [0]:
gc = get_gspread_client_from_secret_old()


# Check for any new requests

In [0]:
### OPEN THE CENTRALIZED SHEET
sh = gc.open_by_url(CENTRALIZED_SHEET_URL)
tab = sh.worksheet("Sheet1")

all_rows = tab.get_all_values()
headers = all_rows[0]
vals = all_rows[1:]


# ONLY LOOK FOR REQUESTS THAT WERE JUST SUBMITTED
requests = spark.createDataFrame(vals, schema=headers).filter(col("Status") == "SUBMITTED")


In [0]:
requests.display()

In [0]:
if requests.count()<1:
    dbutils.notebook.exit("Nothing to update here...")

## Filter each to different categories based on use case

In [0]:
publishing = requests.filter(col("LocType") == "Publishing").toPandas()
ingame = requests.filter(col("LocType") == "InGame").toPandas()

# we are gonna do we need to further generalize the
marketing = requests.filter(col("LocType") == "Marketing").toPandas()

#generic = requests.filter(col("LocType")!= "Publishing" or col("LocType") != "InGame").toPandas()

#### Run for Publishing

In [0]:

# Run publishing

#TODO: Test this for recent submission (9/18/25)
pub_outputs = []
"""
for idx, row in publishing.iterrows():
    pub_output = dbutils.notebook.run(DIR+"Publishing",10000, row.to_dict())
    output = json.loads(output)
    row['Status'] = output['status']
    row['RunID'] = output['run_id']
    row['LastStatusUpdate'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    pub_outputs.append(row)
"""

#### Run for InGame

In [0]:
# Run ingame

in_game_outputs = []
for idx, row in ingame.iterrows():
   #print(row)
    output = dbutils.notebook.run(DIR+"InGame Localizer",10000, row.to_dict()))
    output = json.loads(output)

    row['Status'] = output['status']
    row['RunID'] = output['run_id']
    row['LastStatusUpdate'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    in_game_outputs.append(row)


#### Run for marketing

In [0]:
# Run marketing
marketing_outputs = []
for idx, row in marketing.iterrows():
   
    # run the marketing notebook for each row
    output = dbutils.notebook.run(DIR+"Generic Localizer", 10000,row.to_dict())

    output = json.loads(output)
    row['Status'] = output['status']
    row['RunID'] = output['run_id']
    row['LastStatusUpdate'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    marketing_outputs.append(row)

In [0]:
# Run generic
generic_outputs = []
"""
for idx, row in marketing.iterrows():
   
    output = dbutils.notebook.run(DIR+"Generic Localizer", 10000,row.to_dict())

    output = json.loads(output)
    row['Status'] = output['status']
    row['RunID'] = output['run_id']
    row['LastStatusUpdate'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    generic_outputs.append(row)"""

In [0]:
## Ok, write all together.

updates = []
if len(pub_outputs)>0:
    updates.append(pd.DataFrame(pub_outputs))
if len(in_game_outputs)>0:
    updates.append(pd.DataFrame(in_game_outputs))   
if len(marketing_outputs)>0:
    updates.append(pd.DataFrame(marketing_outputs))
if len(generic_outputs)>0:
    updates.append(pd.DataFrame(generic_outputs))

if len(updates)>1:
    updates = pd.concat(updates)
if len(updates)==1:
    updates = updates[0] 
if len(updates)==0:
    print("nothing to update!!")
    dbutils.notebook.exit("Nothing to see here")

## Update the cells in the centralized tracking sheet

In [0]:
updates

In [0]:
for i, row in updates.iterrows():
    cell = tab.find(row['RowFingerprint'])
    status_range = f"I{cell.row}" # status
    last_updated = f"J{cell.row}" # last updated
    run_id = f"L{cell.row}" # run_id 


    tab.update_acell(status_range,row['Status'])
    tab.update_acell(last_updated,row['LastStatusUpdate'])
    tab.update_acell(run_id,row['RunID'])

